In [ ]:
!pip install numpy pandas scikit-learn requests

  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
  Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl (11.3 MB)
  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl (8.9 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
  Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl (41.3 MB)


You should consider upgrading via the 'c:\Users\montu\Desktop\examen\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
# 1. Importar librerías
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [8]:
# 2. Simular datos de sensores para entrenamiento
np.random.seed(42)
n = 200
data = {
    'temperatura': np.random.randint(20, 35, n),
    'latido': np.random.randint(60, 120, n),
    'movimiento': np.random.randint(0, 2, n),
    'humedad': np.random.randint(40, 80, n),
    'depredador': np.random.randint(0, 2, n),
    'riesgo': np.random.randint(0, 2, n)
}
df = pd.DataFrame(data)
df.head()

,temperatura,latido,movimiento,humedad,depredador,riesgo
0,26,96,1,62,1,0
1,23,82,0,48,0,1
2,32,98,1,51,1,1
3,34,104,0,40,0,1
4,30,74,1,40,0,0


In [12]:
# 3. Entrenar modelo de clasificación con todos los sensores
X = df[['temperatura', 'latido', 'movimiento', 'humedad', 'depredador']]
y = df['riesgo']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(f"\n✅ Modelo entrenado con {len(X.columns)} características:")
print(f"   - Temperatura")
print(f"   - Latido cardíaco")
print(f"   - Movimiento")
print(f"   - Humedad")
print(f"   - Presencia de depredador")

              precision    recall  f1-score   support

           0       0.35      0.60      0.44        15
           1       0.57      0.32      0.41        25

    accuracy                           0.42        40
   macro avg       0.46      0.46      0.42        40
weighted avg       0.49      0.42      0.42        40


✅ Modelo entrenado con 5 características:
   - Temperatura
   - Latido cardíaco
   - Movimiento
   - Humedad
   - Presencia de depredador


In [15]:
# 4. Usar el modelo con datos reales de Firebase (Firestore)
import requests

url = "https://firestore.googleapis.com/v1/projects/mbe1-7a7b0/databases/(default)/documents/sensores"
response = requests.get(url)
data = response.json()

print("🔍 Analizando datos reales de Firebase...\n")
print(f"📊 Total de documentos encontrados: {len(data.get('documents', []))}\n")

for doc in data.get('documents', []):
    try:
        if 'fields' not in doc:
            print(f"⚠️ Documento sin campo 'fields', saltando...\n")
            continue
            
        fields = doc['fields']
        
        def obtener_valor(campo, default=None):
            if campo not in fields:
                return default
            valor = fields[campo]
            if isinstance(valor, dict):
                return int(valor.get('integerValue', valor.get('stringValue', default)))
            return int(valor) if valor is not None else default
        
        temperatura = obtener_valor('valor', 25)
        latido = obtener_valor('latido', 80)
        movimiento = obtener_valor('movimiento', 0)
        humedad = obtener_valor('humedad', 60)
        depredador = obtener_valor('depredador', 0)
        
        entrada_df = pd.DataFrame({
            'temperatura': [temperatura],
            'latido': [latido],
            'movimiento': [movimiento],
            'humedad': [humedad],
            'depredador': [depredador]
        })
        
        prediccion = clf.predict(entrada_df)[0]
        probabilidad = clf.predict_proba(entrada_df)[0]
        
        print(f"📈 Datos: Temp={temperatura}°C | Latido={latido}bpm | Mov={movimiento} | Hum={humedad}% | Dep={depredador}")
        print(f"   🎯 Predicción: {'⚠️ RIESGO' if prediccion == 1 else '✅ Normal'} (Confianza: {probabilidad[prediccion]*100:.1f}%)\n")
        
    except Exception as e:
        print(f"⚠️ Error procesando documento: {e}\n")

🔍 Analizando datos reales de Firebase...

📊 Total de documentos encontrados: 11

📈 Datos: Temp=31°C | Latido=67bpm | Mov=0 | Hum=53% | Dep=0
   🎯 Predicción: ✅ Normal (Confianza: 69.0%)

📈 Datos: Temp=29°C | Latido=98bpm | Mov=1 | Hum=71% | Dep=0
   🎯 Predicción: ⚠️ RIESGO (Confianza: 63.0%)

📈 Datos: Temp=21°C | Latido=68bpm | Mov=1 | Hum=62% | Dep=0
   🎯 Predicción: ✅ Normal (Confianza: 88.0%)

📈 Datos: Temp=33°C | Latido=89bpm | Mov=0 | Hum=52% | Dep=1
   🎯 Predicción: ✅ Normal (Confianza: 75.0%)

📈 Datos: Temp=25°C | Latido=78bpm | Mov=1 | Hum=79% | Dep=1
   🎯 Predicción: ⚠️ RIESGO (Confianza: 52.0%)

📈 Datos: Temp=28°C | Latido=110bpm | Mov=1 | Hum=72% | Dep=0
   🎯 Predicción: ✅ Normal (Confianza: 55.0%)

📈 Datos: Temp=26°C | Latido=108bpm | Mov=0 | Hum=66% | Dep=1
   🎯 Predicción: ⚠️ RIESGO (Confianza: 60.0%)

📈 Datos: Temp=29°C | Latido=108bpm | Mov=0 | Hum=74% | Dep=0
   🎯 Predicción: ⚠️ RIESGO (Confianza: 61.0%)

📈 Datos: Temp=21°C | Latido=109bpm | Mov=0 | Hum=43% | Dep=0
   

In [16]:
# 5. Enviar predicciones de riesgo a Firebase (colección alertas)
import json
from datetime import datetime, timezone

# Configuración de Firebase
PROJECT_ID = "mbe1-7a7b0"
COLLECTION_ALERTAS = "alertas"

# Función para enviar alerta a Firebase
def enviar_alerta_firebase(temperatura, latido, movimiento, prediccion_riesgo):
    """
    Envía una alerta a Firebase Firestore usando la API REST
    NOTA: Requiere que las reglas de Firestore permitan escritura sin autenticación
    (solo para desarrollo)
    """
    url = f"https://firestore.googleapis.com/v1/projects/{PROJECT_ID}/databases/(default)/documents/{COLLECTION_ALERTAS}"
    
    # Preparar timestamp en formato correcto para Firebase
    # Firebase requiere formato: "2025-11-21T20:00:00Z" o con timezone
    now = datetime.now(timezone.utc)
    timestamp_str = now.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + "Z"
    
    datos = {
        "fields": {
            "temperatura": {"integerValue": str(int(temperatura))},
            "latido": {"integerValue": str(int(latido))},
            "movimiento": {"integerValue": str(int(movimiento))},
            "riesgo": {"integerValue": str(int(prediccion_riesgo))},
            "tipo": {"stringValue": "prediccion_ml"},
            "timestamp": {"timestampValue": timestamp_str},
            "mensaje": {
                "stringValue": "Riesgo detectado" if prediccion_riesgo == 1 else "Estado normal"
            }
        }
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    try:
        response = requests.post(url, json=datos, headers=headers)
        if response.status_code == 200:
            print(f"✅ Alerta enviada: Riesgo={prediccion_riesgo}, Temp={temperatura}°C")
            return True
        elif response.status_code == 403:
            print(f"⚠️ Error 403: Permisos denegados. Verifica las reglas de Firestore.")
            print(f"   Para desarrollo, las reglas deben permitir escritura:")
            print(f"   allow write: if true;")
            return False
        else:
            print(f"❌ Error al enviar: {response.status_code}")
            print(f"   Respuesta: {response.text[:200]}")
            return False
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Procesar datos y enviar alertas solo cuando hay riesgo
print("🔍 Analizando datos y enviando alertas a Firebase...\n")
print("⚠️ NOTA: Asegúrate de que las reglas de Firestore permitan escritura\n")
alertas_enviadas = 0
errores = 0

for doc in data.get('documents', []):
    try:
        if 'fields' not in doc:
            continue
            
        fields = doc['fields']
        
        def obtener_valor(campo, default=None):
            if campo not in fields:
                return default
            valor = fields[campo]
            if isinstance(valor, dict):
                return int(valor.get('integerValue', valor.get('stringValue', default)))
            return int(valor) if valor is not None else default
        
        temperatura = obtener_valor('valor', 25)
        latido = obtener_valor('latido', 80)
        movimiento = obtener_valor('movimiento', 0)
        humedad = obtener_valor('humedad', 60)
        depredador = obtener_valor('depredador', 0)
        
        entrada_df = pd.DataFrame({
            'temperatura': [temperatura],
            'latido': [latido],
            'movimiento': [movimiento],
            'humedad': [humedad],
            'depredador': [depredador]
        })
        prediccion = clf.predict(entrada_df)[0]
        
        if prediccion == 1:
            if enviar_alerta_firebase(temperatura, latido, movimiento, prediccion):
                alertas_enviadas += 1
            else:
                errores += 1
    except Exception as e:
        print(f"⚠️ Error procesando documento: {e}")
        errores += 1

print(f"\n📊 Resumen:")
print(f"   ✅ Alertas enviadas: {alertas_enviadas}")
print(f"   ❌ Errores: {errores}")
print(f"\n💡 Si hay errores 403, configura las reglas de Firestore:")
print("   rules_version = '2';")
print("   service cloud.firestore {")
print("     match /databases/{database}/documents {")
print("       match /{document=**} {")
print("         allow read, write: if true;")
print("       }")
print("     }")
print("   }")


🔍 Analizando datos y enviando alertas a Firebase...

⚠️ NOTA: Asegúrate de que las reglas de Firestore permitan escritura

✅ Alerta enviada: Riesgo=1, Temp=29°C
✅ Alerta enviada: Riesgo=1, Temp=25°C
✅ Alerta enviada: Riesgo=1, Temp=26°C
✅ Alerta enviada: Riesgo=1, Temp=29°C
✅ Alerta enviada: Riesgo=1, Temp=21°C
✅ Alerta enviada: Riesgo=1, Temp=33°C

📊 Resumen:
   ✅ Alertas enviadas: 6
   ❌ Errores: 0

💡 Si hay errores 403, configura las reglas de Firestore:
   rules_version = '2';
   service cloud.firestore {
     match /databases/{database}/documents {
       match /{document=**} {
         allow read, write: if true;
       }
     }
   }
